In [30]:
#importing stuff
from io import StringIO
import numpy as np
import os

In [ ]:
sample_rate = 32 #Hz
sensor_min = -1.5 #g
sensor_max =  1.5 #g
sample_res = 63

def normalData(_data):
    data = -sensor_min + (_data/sample_res) * (sensor_max - sensor_min)
    return data

In [31]:
#%ls HMP_Dataset/Walk/
def parseAdlTxt(file):
    s = "HMP_Dataset/Walk/Accelerometer-2011-03-24-09-51-07-walk-f1.txt"
    data = np.genfromtxt(s, dtype=np.float32, delimiter=" ")
    #normalize data here
    return data

In [47]:
def getRecordsFromTxt(folderPath):
    for root, dirs, files in os.walk("HMP_Dataset/Walk/", topdown=False):
        prev_shape = None
        records = [] #append a list
        for name in files:
            data = parseAdlTxt(os.path.join(root, name))

            if(prev_shape != None and prev_shape != data.shape):
                print("Error: data sample length not uniform")
                exit(-1)
            else:
                prev_shape = data.shape

            records.append(data)
            #print(name, data.shape)
    return records

Accelerometer-2011-03-24-09-51-07-walk-f1.txt (1170, 3)
Accelerometer-2011-03-24-09-52-11-walk-f1.txt (1170, 3)
Accelerometer-2011-03-24-10-58-34-walk-f1.txt (1170, 3)
Accelerometer-2011-03-24-11-20-24-walk-f1.txt (1170, 3)
Accelerometer-2011-03-24-11-21-13-walk-f1.txt (1170, 3)
Accelerometer-2011-03-29-16-16-01-walk-f1.txt (1170, 3)
Accelerometer-2011-04-05-18-30-57-walk-f1.txt (1170, 3)
Accelerometer-2011-04-11-11-55-57-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-08-29-07-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-09-35-03-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-10-29-28-walk-m1.txt (1170, 3)
Accelerometer-2011-05-30-20-51-03-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-20-57-19-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-21-01-39-walk-f1.txt (1170, 3)
Accelerometer-2011-05-30-21-34-15-walk-m2.txt (1170, 3)
Accelerometer-2011-05-30-21-39-55-walk-m2.txt (1170, 3)
Accelerometer-2011-05-30-21-44-35-walk-m2.txt (1170, 3)
Accelerometer-2011-05-31-14-54-41-walk-f1.txt (1

In [22]:
print(data.shape)

(1170, 3)
